# ポイントクラウドを用いた平面の検出


Q16: ベクトル $\bf a$ のノルム（長さ）を $\|\bf a\|$と記す．また，ベクトル $\bf a$ と $\bf b$の内積を ${\bf a}\cdot{\bf b}$，外積を ${\bf a}\times{\bf b}$ と記す．__これらの記法を用いて__，「位置ベクトル ${\bf p}_0$，${\bf p}_1$，${\bf p}_2$ の3点を通る平面」と「位置ベクトル $\bf p$ の点」の間の距離 $d$ を求める公式を作れ．また，この公式を図を用いて解説せよ．

## RealSenseで平面を撮像してください．
> 机や壁と多面体など，なるべく2つ以上の面が広く写るように撮影しましょう．

> このファイルの内容は，★印まで pointcloud_rs.ipynb と同じです．★印まで同様に作業を進めてください． 

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
%matplotlib inline

# Configure color and depth to run at VGA resolution at 30 frames per second
config = rs.config()
config.enable_stream(rs.stream.depth)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

## カラー画像と深度画像を取得して表示・保存します．
スペースキーを押す毎に画像が連番で保存されます．'q'を押すと終了します．最後に保存したデータが点群の作成に使われます．

In [ ]:
# Start streaming
pipeline = rs.pipeline()
profile = pipeline.start(config)

# Get camera parameters
intr = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
scale = config.resolve(rs.pipeline_wrapper(pipeline)).get_device().first_depth_sensor().get_depth_scale()

print("focal length(x) in pixels = ", intr.fx)
print("focal length(y) in pixels = ", intr.fy)
print("image height = ", intr.height)
print("image width = ", intr.width)
print("ppx = ", intr.ppx)
print("ppy = ", intr.ppy)

# Create a camera alignment object (depth aligned to color)
align = rs.align(rs.stream.color)
max_depth = 2.0 / scale # Zeros out for any depth greater than 2.0 meters

# Display and save images
print("Press [SPACE] to save images (png) and depth data (npy).")
print("Press 'q' to stop.")
nsaved = 0
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        bgr = np.asanyarray(color_frame.get_data())
        depth = np.asanyarray(depth_frame.get_data())
        depth[depth > max_depth] = 0 # Zeros out

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U), 
                                           cv2.COLORMAP_JET)
  
        images = np.hstack((bgr, depth_colormap))
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        
        key = cv2.waitKey(33)
        if key == ord(' '):
            Z = depth * scale * 1e+3 # unit in mm
            color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            
            # Save images
            cv2.imwrite('color{:02d}pc.png'.format(nsaved), bgr)
            cv2.imwrite('depth{:02d}pc.png'.format(nsaved), depth_colormap)
            np.save('Z{:02d}pc.npy'.format(nsaved), Z)
            
            print("color image and depth data are saved ({:02d})".format(nsaved))
            nsaved += 1

        elif key == ord('q'):
            if nsaved == 0:
                Z = depth * scale * 1e+3 # unit in mm
                color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            cv2.destroyAllWindows()
            break
        
finally:
    # Stop streaming
    pipeline.stop()

## 逆透視変換でポイントクラウドを作りましょう．
カラー画像および点群を作成する深度データを可視化します．

In [ ]:
from PIL import Image
height, width, _ = color.shape
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.imshow(color)
plt.subplot(1,2,2)
plt.imshow(Z, cmap="gray")

### 各画素にuとvの座標を設定します．座標系は[予習事項](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/Document/preparation.pdf)の図1です．

In [ ]:
# 主点を設定します．
cx, cy = intr.ppx, intr.ppy # width*0.5, height*0.5
j_to_u = lambda j: -(j - cx)
i_to_v = lambda i: -(i - cy)
# 画像平面の座標を設定します．
u, v = np.meshgrid(j_to_u(np.arange(width)), i_to_v(np.arange(height)))
print(u, u.shape)
print(v, v.shape)

### Z, u, vが与えられたとき，X[mm]とY[mm]を計算する関数を作りましょう．[ヒント：予習事項の問2](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/Document/preparation.pdf)

In [ ]:
focal_length = [intr.fx, intr.fy] # in pixels
def Zuv_to_XY(Z, u, v, f=focal_length):
    ### X = ________   # Z, u, v, f[0], f[1] から必要なものを使って計算する
    ### Y = ________   # Z, u, v, f[0], f[1] から必要なものを使って計算する
    return X, Y

In [ ]:
# Z, u, v から X, Y を計算します．
X, Y = Zuv_to_XY(Z, u, v)

## ★ポイントクラウドを可視化して観察しましょう．
> マウスで視点や拡大・縮小をコントロールできます．描画領域右上にある機能も活用しましょう．

In [ ]:
nd = np.count_nonzero(Z)
n = 30000
p = np.random.choice(nd, min(n,nd), replace=False)
print("%d out of %d points are displayed." % (n, nd))

import plotly.graph_objs as go
rgb = color[Z>0][p] # * 1.5 # brighter

trace = go.Scatter3d(x=X[Z>0][p], y=Y[Z>0][p], z=Z[Z>0][p], mode='markers',
                     marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(rgb[:,0], rgb[:,1], rgb[:,2])],
                                opacity=0.8))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)
fig.show()

## 最も多い点で表された平面の法線と通る点を推定する関数`DetectPlane`を定義します．
[[Tarsha-Kurdi+08]](https://halshs.archives-ouvertes.fr/halshs-00278397/document)によるRANSACアルゴリズムを参考に実装したものです．

入力:
> `points`:  n行3列のNumPy配列．nは点の数，各列はX,Y,Z座標を表します．

> `n_trials`:  試行回数（規定値30回）

> `th`:  面からの距離の閾値（規定値3mm）

出力
> `plane`:  推定した平面のパラメタ．

>> `plane["normal"]`:  法線ベクトル

>> `plane["p3idx"]`:  通る3点の番号．面は `points[plane["p3idx"][0]]`，`points[plane["p3idx"][1]]`，`points[plane["p3idx"][2]]`の3点を通ります．

Q17: [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus)とは何か．原理と特長を述べよ．

Q18: 関数`DetectPlane`が平面の法線と通る点を推定する仕組みを解説せよ．


In [ ]:
def DetectPlane(points, n_trials=30, th=3):

    # initial settings
    plane = dict(normal=None, p3idx=None)
    n_max, dev_min = 0, float("inf")

    for i in range(n_trials):
        # randomly pick up three points
        p3idx = np.random.choice(points.shape[0], 3, replace=False)

        # compute a unit normal vector
        normal = np.cross(points[p3idx[1]] - points[p3idx[0]], points[p3idx[2]] - points[p3idx[0]])
        normal = normal / np.linalg.norm(normal)

        # compute distances from the plane with a point p3idx[0] and the normal vector
        distances = np.abs(np.dot(points - points[p3idx[0],:], normal))

        # find the neighboring points to the plane
        pidx_neighbors = np.where(distances < th)[0]
        num_neighbors = len(pidx_neighbors)
        deviation = np.std(distances[pidx_neighbors])

        # check if the plane is better than the current estimate
        if num_neighbors > n_max or (num_neighbors == n_max and deviation < dev_min):
            n_max, dev_min = num_neighbors, deviation
            plane["normal"], plane["p3idx"] = normal, p3idx

    return plane

### ポイントクラウドに適用して，最大の平面を検出します．

Q19: 表示されるヒストグラムは何を表しているか．このヒストグラムから何がわかるか．

In [ ]:
points = np.vstack((X[Z>0],Y[Z>0],Z[Z>0])).T
plane1 = DetectPlane(points, n_trials=100, th=5)

print("Estimated unit normal vector =", plane1["normal"])
distances1 = np.abs(np.dot(points - points[plane1["p3idx"][0],:], plane1["normal"]))
import matplotlib.pyplot as plt
_ = plt.hist(distances1, bins=50)
plt.xlabel("Distance [mm]")

### 検出した平面に近い点を着色して表示します．

「検出」の誤りは2種類あります．誤検出（false positive detection）と検出漏れ（false negative detection）です．平面の検出では，どのような原因で誤検出や検出漏れが起きるでしょうか．実験的に具体例を示しながら考察してください．

Q20: 入力の `n_trials` や `th` の値が大きい・小さいと，`DetectPlane`による平面の検出結果はどうなるか．表示される図を用いて説明せよ．また，その結果になる原因を考察せよ．
> 前のセル（`plane1 = ...`）で入力を変えて実行し，次のセル（`# plot the neighboring ...`）で表示・観察する．

> 入力が同じでも`DetectPlane`は実行する毎に異なる結果を出力することがある．反復して観察すること． 

In [ ]:
# plot the neighboring points to the plane within 10mm
disp_mm = 10.
pidx_on_plane1 = np.where(distances1 < disp_mm)[0]
p1 = np.intersect1d(p, pidx_on_plane1)
print("Points within %2.0f mm of the plane are shown in green." % (disp_mm))
xyz = points[p1,:]

trace_p1 = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                           marker=dict(size=2, color='rgb(0,255,0)',
                           opacity=0.05))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace, trace_p1], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)

fig.show()

### 検出した平面に近い点を除いたポイントクラウドから，再び平面を検出します．

In [ ]:
points1 = np.delete(points, pidx_on_plane1, axis=0)

plane2 = DetectPlane(points1, n_trials=100, th=5)

print("Estimated unit normal vector =", plane2["normal"])
distances2 = np.abs(np.dot(points - points[plane2["p3idx"][0],:], plane2["normal"]))
import matplotlib.pyplot as plt
_ = plt.hist(distances2, bins=50)
plt.xlabel("Distance [mm]")

### 検出した平面に近い点を着色して表示します．

どのような平面が2つ目の平面として検出されますか．前のセル（`points1 = ...`）と次のセル（`# plot the neighboring ...`）の実行を何度か繰り返して観察してください．

In [ ]:
# plot the neighboring points to the plane within 10mm
disp_mm = 10.
pidx_on_plane2 = np.where(distances2 < disp_mm)[0]
p2 = np.intersect1d(p, pidx_on_plane2)
print("Points within %2.0f mm of the 2nd plane are shown in red." % (disp_mm))
xyz = points[p2,:]

trace_p2 = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                           marker=dict(size=2, color='rgb(255,0,0)',
                           opacity=0.05))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace, trace_p1, trace_p2], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)

fig.show()